In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [3]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [4]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [5]:
# how many columns in a dataset
len(df.columns)

19

In [6]:
df.shape[1]

19

In [7]:
# compute the duration
df['duration'] = df['tpep_dropoff_datetime']-df['tpep_pickup_datetime']

In [8]:
# compute the standard deviation of duration
print(df['duration'].std())

0 days 00:42:35.661074517


In [10]:
original_rows = df.shape[0]

In [12]:
# keep the duration between 1min and 60min
df = df[(df['duration'] >= pd.Timedelta(minutes=1)) &
        (df['duration'] <= pd.Timedelta(minutes=60))]

In [13]:
print(df.shape[0]/original_rows)

0.9812202822125979


In [14]:
df['PULocationID'] = df['PULocationID'].astype(str)
df['DOLocationID'] = df['DOLocationID'].astype(str)
data_dicts = df[['PULocationID', 'DOLocationID']].to_dict(orient='records')

In [15]:
vec = DictVectorizer(sparse=True)
X = vec.fit_transform(data_dicts)

In [16]:
print(X.shape)

(3009173, 515)


In [17]:
# Convert to numeric duration in minutes
df['duration_minutes'] = df['duration'] / pd.Timedelta(minutes=1)
y = df['duration_minutes'] 

In [18]:
# Fit Linear Regression
model = LinearRegression()
model.fit(X, y)
y_pred = model.predict(X)
rmse = np.sqrt(mean_squared_error(y, y_pred))
print("RMSE on test set:", rmse)

RMSE on test set: 7.6492610279057605


In [19]:
df_val = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [20]:
df_val.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00


In [21]:
# Compute duration for validation set
df_val['duration'] = df_val['tpep_dropoff_datetime'] - df_val['tpep_pickup_datetime']

# Filter to valid trips between 1 and 60 minutes
df_val = df_val[(df_val['duration'] >= pd.Timedelta(minutes=1)) &
                (df_val['duration'] <= pd.Timedelta(minutes=60))]

# Convert to numeric duration in minutes
df_val['duration_minutes'] = df_val['duration'] / pd.Timedelta(minutes=1)
y_val = df_val['duration_minutes']

In [22]:
# Convert categorical features to string
df_val['PULocationID'] = df_val['PULocationID'].astype(str)
df_val['DOLocationID'] = df_val['DOLocationID'].astype(str)

# Turn into list of dicts
val_dicts = df_val[['PULocationID', 'DOLocationID']].to_dict(orient='records')

# Transform using the already-fit DictVectorizer
X_val = vec.transform(val_dicts)  # DO NOT use fit_transform here!

In [11]:
# Predict using the trained model
y_val_pred = model.predict(X_val)

# Compute RMSE
rmse_val = np.sqrt(mean_squared_error(y_val, y_val_pred))
print("Validation RMSE (Feb 2023):", rmse_val)

Validation RMSE (Feb 2023): 7.812140507792699
